In [56]:
import pandas as pd
import re
import numpy as np

In [57]:
stats = pd.read_excel('COMBINED_NORMAL_ADVANCED_STATS_1984-85_to_2019-20_ver2.xlsx', sheet_name=1)

In [58]:
stats['error_in_name'] = stats['slug'].apply(lambda x: x is np.nan)

In [59]:
stats_need_fix = stats.loc[stats['error_in_name'] == True]

In [60]:
stats_need_fix['Year'] = '2019-20'
name = stats_need_fix['name'].apply(lambda x: re.sub(r"\\.+", "", x))
slug = stats_need_fix['name'].apply(lambda x: re.sub(r".+\\", "", x))
stats_need_fix['name'], stats_need_fix['slug'] = name, slug

<ipython-input-60-5b3e32999086>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stats_need_fix['Year'] = '2019-20'
<ipython-input-60-5b3e32999086>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stats_need_fix['name'], stats_need_fix['slug'] = name, slug


In [61]:
stats = stats.loc[stats['error_in_name'] == False].append(stats_need_fix).drop(columns=['error_in_name'])

In [62]:
salaries = pd.read_excel('salaries_1985to2020_final.xlsx')
salary_cap = pd.read_excel('salary_caps.xlsx')

In [63]:
merged = salaries.merge(salary_cap[['season', 'salary_cap']], on='season')

In [64]:
merged['cap_space_usage'] = merged['salary'] / merged['salary_cap']

In [65]:
stats['end season'] = stats['Year'].apply(lambda x: '19' + x[-2:] if int(x[-2:]) >= 84 else '20' + x[-2:])

In [66]:
stats['key'] = stats['name'] + stats['end season']

In [67]:
merged['season_end'] = merged['season_end'].apply(lambda x: str(x))

In [68]:
merged['key'] = merged['names'] + merged['season_end']

In [69]:
final_table = stats.merge(merged, on='key')

In [70]:
final_table.columns

Index(['name', 'positions', 'age', 'team_x', 'games_played', 'minutes_played',
       'player_efficiency_rating', 'true_shooting_percentage',
       'three_point_attempt_rate', 'free_throw_attempt_rate',
       'offensive_rebound_percentage', 'defensive_rebound_percentage',
       'total_rebound_percentage', 'assist_percentage', 'steal_percentage',
       'block_percentage', 'turnover_percentage', 'usage_percentage',
       'offensive_win_shares', 'defensive_win_shares', 'win_shares',
       'win_shares_per_48_minutes', 'offensive_box_plus_minus',
       'defensive_box_plus_minus', 'box_plus_minus',
       'value_over_replacement_player', 'is_combined_totals', 'key', 'Year',
       'slug', 'end season', 'names', 'player_id', 'salary', 'season',
       'season_end', 'season_start', 'team_y', 'salary_cap',
       'cap_space_usage'],
      dtype='object')

In [71]:
final_table.to_csv('bbr_normal_advanced_cap_usage.csv')